# Check funding payment entities

* Query positions to ensure `netFunding` on positions matches the total of funding payments


In [3]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px

load_dotenv()

pio.renderers.default = 'notebook'

In [4]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-goerli-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [5]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)
convertBytes = lambda x: bytearray.fromhex(x[2:]).decode().replace('\x00', '')

def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [11]:
position_query = gql("""
query positions(
    $last_id: ID!
  ) {
    futuresPositions(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      asset
      initialMargin
      size
      entryPrice
      avgEntryPrice
      lastPrice
      exitPrice
      pnl
      pnlWithFeesPaid
      openTimestamp
      timestamp
      closeTimestamp
      feesPaid
      netFunding
      isLiquidated
      isOpen
      margin
      netTransfers
      totalDeposits
      totalVolume
    }  
}
""")

funding_query = gql("""
query funding(
    $last_id: ID!
  ) {
    fundingPaids(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      timestamp
      account
      positionId
      marketKey
      asset
      amount
    }  
}
""")


### Run a query

In [12]:
position_params = {
    'last_id': ''
}

position_cols = [
    'avgEntryPrice',
    'entryPrice',
    'exitPrice',
    'feesPaid',
    'initialMargin',
    'lastPrice',
    'margin',
    'netFunding',
    'netTransfers',
    'pnl',
    'pnlWithFeesPaid',
    'size',
    'totalDeposits',
    'totalVolume'
]

position_bytes_cols = [
    'asset'
]

df_position = await run_recursive_query(position_query, position_params, 'futuresPositions')
df_position = df_position.replace({None: np.nan})
df_position = clean_df(df_position, decimal_cols=position_cols, bytes_cols=position_bytes_cols)


In [13]:
df_position

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,timestamp,closeTimestamp,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume
0,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0,0xf526eb7d2d4445fa8d258959a000400cca4a93f4,sETH,10,0,1203.02,1203.02,1679.2731,NaN,0,...,1678731776,NaN,0,0,False,True,6.009282071650675695,423252.965372126957228405,3389114.776901719498147498,0
1,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x100,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,sETH,109.786879000577615685,0,1574.865430172633161956,1574.865430172633161956,1575.272931567707450281,1575.272931567707450281,-0.258858677303379986,...,1678124704,1678124704,1.50087058828004891,-0.023457078348741274,False,False,108.003692656645445514,0,0,2001.074498147630755744
2,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x101,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,sETH,108.003692656645445514,0,1573.311424227520186542,1573.311424227520186542,1573.353924401059118668,1573.353924401059118668,0.026997548573904826,...,1678125106,1678125106,1.499144492623722242,0.015513800139983113,False,False,106.547059512735611212,0,0,1998.868322681154292106
3,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x102,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,sETH,206.489856327447689548,0,1573.802926509095817218,1573.802926509095817218,1573.744176676525028046,1573.744176676525028046,0.037319881931439978,...,1678125434,1678125434,1.499562002292405904,-0.022563283148981909,False,False,205.005050923937741714,0,0,1999.428443017185019991
4,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x103,0x48914229dedd5a9922f44441ffccfc2cb7856ee9,sETH,1058748.06401214770766184,0,1538.191756184197533556,1538.191756184197533556,1539.320853256986952571,1539.320853256986952571,-5024.917077925409809461,...,1678126648,1678126648,10319.253841824367309145,-202.567329752552979642,False,False,1043201.325762645377563593,0,0,13696117.049092679321682272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,0xe140356ab1f0558e020610c9c6bccdaa4fdde2f4-0xe,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,AVAX,100.931047147294703616,0,16.437574785163995271,16.437574785163995271,16.301796175763910923,16.301796175763910923,8.337586637048125971,...,1678022338,1678022338,1.608308366304564033,0.37708751183721088,False,False,200.321282774284551413,93.493074109825794048,302.694167848624242688,2010.385457880705043137
262,0xe140356ab1f0558e020610c9c6bccdaa4fdde2f4-0xf,0x76758524bbc67e54e7f6c6967ab2f7be1d1c6ad8,AVAX,102275.512203303681399991,0,16.279592169726548985,16.279592169726548985,15.357092,NaN,409928.069568794261301797,...,1678337618,1678337618,2451.270036704923237227,-509752.311735393019464561,True,False,0,0,0,1209391.697254102697689973
263,0xeff373efbd352531f680575796ae14764be0391b-0x0,0x3239a95a9262034ca28b9a03133775f716f119f8,APE,100,0,5.8832655,5.8832655,5.2324,NaN,0,...,1677260984,NaN,0,0,False,True,0,12.340763289771703651,100,0
264,0xeff373efbd352531f680575796ae14764be0391b-0x1,0x3239a95a9262034ca28b9a03133775f716f119f8,APE,99,0,5.892503334748160722,5.892503334748160722,5.238321584526666665,5.238321584526666665,-11.10665110732587533,...,1677260694,1677260694,1.191199681991835758,-0.042912500453992563,False,False,87.659236710228296349,0,0,188.978351770370583817


In [14]:
df_position.loc[df_position['netFunding'].abs() > 0, :].head()

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,timestamp,closeTimestamp,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume
1,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x100,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,sETH,109.786879000577615685,0,1574.865430172633161956,1574.865430172633161956,1575.272931567707450281,1575.272931567707450281,-0.258858677303379986,...,1678124704,1678124704,1.50087058828004891,-0.023457078348741274,False,False,108.003692656645445514,0,0,2001.074498147630755744
2,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x101,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,sETH,108.003692656645445514,0,1573.311424227520186542,1573.311424227520186542,1573.353924401059118668,1573.353924401059118668,0.026997548573904826,...,1678125106,1678125106,1.499144492623722242,0.015513800139983113,False,False,106.547059512735611212,0,0,1998.868322681154292106
3,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x102,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,sETH,206.489856327447689548,0,1573.802926509095817218,1573.802926509095817218,1573.744176676525028046,1573.744176676525028046,0.037319881931439978,...,1678125434,1678125434,1.499562002292405904,-0.022563283148981909,False,False,205.005050923937741714,0,0,1999.428443017185019991
4,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x103,0x48914229dedd5a9922f44441ffccfc2cb7856ee9,sETH,1058748.06401214770766184,0,1538.191756184197533556,1538.191756184197533556,1539.320853256986952571,1539.320853256986952571,-5024.917077925409809461,...,1678126648,1678126648,10319.253841824367309145,-202.567329752552979642,False,False,1043201.325762645377563593,0,0,13696117.049092679321682272
5,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x104,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,sETH,9664.824159495873411986,0,1504.105123155734919529,1504.105123155734919529,1575.164551781495849906,1575.164551781495849906,4470.424448861139850202,...,1678126690,1678126690,96.860069843607496251,2.779722900425607077,False,False,14041.168261413831373014,0,0,193720.13968721499250282


## Funding Payments

In [17]:
funding_params = {
    'last_id': ''
}

funding_decimal_cols = [
    'amount',
]

funding_bytes_cols = [
    'marketKey',
    'asset'
]

df_funding = await run_recursive_query(funding_query, funding_params, 'fundingPaids')
df_funding = df_funding.replace({None: np.nan})
df_funding = clean_df(df_funding, decimal_cols=funding_decimal_cols, bytes_cols=funding_bytes_cols)


In [18]:
df_funding

,id,timestamp,account,positionId,marketKey,asset,amount
0,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0...,1673579748,0x3239a95a9262034ca28b9a03133775f716f119f8,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0,sETHPERP,sETH,0
1,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0...,1674775952,0x3239a95a9262034ca28b9a03133775f716f119f8,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0,sETHPERP,sETH,0
2,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0...,1671423971,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0,sETHPERP,sETH,0
3,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0...,1671423133,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0,sETHPERP,sETH,0
4,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0...,1672452475,0x126cf7243dc9901f7d8f08bbb0d2be8cf4cf6c97,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0,sETHPERP,sETH,0
...,...,...,...,...,...,...,...
1868,0xeff373efbd352531f680575796ae14764be0391b-0x1...,1677260694,0x3239a95a9262034ca28b9a03133775f716f119f8,0xeff373efbd352531f680575796ae14764be0391b-0x1,sAPEPERP,APE,-0.000004122371994431
1869,0xeff373efbd352531f680575796ae14764be0391b-0x1...,1677260624,0x3239a95a9262034ca28b9a03133775f716f119f8,0xeff373efbd352531f680575796ae14764be0391b-0x1,sAPEPERP,APE,-0.042908378081998132
1870,0xf503e527854b510c1952425d4b61c6bba40028ce-0x0...,1675806594,0x3239a95a9262034ca28b9a03133775f716f119f8,0xf503e527854b510c1952425d4b61c6bba40028ce-0x0,sNEARPERP,NEAR,0
1871,0xf503e527854b510c1952425d4b61c6bba40028ce-0x0...,1677260558,0x3239a95a9262034ca28b9a03133775f716f119f8,0xf503e527854b510c1952425d4b61c6bba40028ce-0x0,sNEARPERP,NEAR,0


In [19]:
df_funding['amount'].sum()

Decimal('-646992.464774960599725677')

In [20]:
df_position['netFunding'].sum()

Decimal('-646992.464774960599725677')

In [23]:
df_funding_summ = df_funding.groupby('positionId')['amount'].sum().reset_index()
df_funding_merge = df_funding_summ.merge(df_position[['id', 'netFunding']], left_on='positionId', right_on='id')

print(f'funding summ: {df_funding_summ.shape[0]}')
print(f'merge summ: {df_funding_merge.shape[0]}')
print(f'matching positions: {df_funding_merge[(df_funding_merge['amount'] == df_funding_merge['netFunding'])].shape[0]}')




funding summ: 254
merge summ: 254


,positionId,amount,id,netFunding
0,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0,0,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x0,0
1,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x100,-0.023457078348741274,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x100,-0.023457078348741274
2,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x101,0.015513800139983113,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x101,0.015513800139983113
3,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x102,-0.022563283148981909,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x102,-0.022563283148981909
4,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x103,-202.567329752552979642,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x103,-202.567329752552979642
...,...,...,...,...
249,0xe140356ab1f0558e020610c9c6bccdaa4fdde2f4-0xe,0.377087511837210880,0xe140356ab1f0558e020610c9c6bccdaa4fdde2f4-0xe,0.37708751183721088
250,0xe140356ab1f0558e020610c9c6bccdaa4fdde2f4-0xf,-509752.311735393019464561,0xe140356ab1f0558e020610c9c6bccdaa4fdde2f4-0xf,-509752.311735393019464561
251,0xeff373efbd352531f680575796ae14764be0391b-0x0,0,0xeff373efbd352531f680575796ae14764be0391b-0x0,0
252,0xeff373efbd352531f680575796ae14764be0391b-0x1,-0.042912500453992563,0xeff373efbd352531f680575796ae14764be0391b-0x1,-0.042912500453992563
